In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# Exercises

1. Read the case, department, and source data into their own spark dataframes.

In [2]:
# specify the data types before creating df
from pyspark.sql.types import StructType, StructField, StringType

schema_src = StructType([
    StructField('source_id', StringType()),
    StructField('source_username', StringType()),
])

In [3]:
# reading data to a spark dataframe
src = spark.read.csv("data/source.csv", sep=",", header=True, schema=schema_src)
dep = spark.read.csv("data/dept.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv("data/case.csv", sep=",", header=True, inferSchema=True)

In [4]:
# looking into first two rows of data
src.show(2), dep.show(2), src.show(2)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
+---------+----------------+
only showing top 2 rows

+---------------+--------------------+----------------------+-------------------+
|  dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+--------------------+----------------------+-------------------+
|311 Call Center|    Customer Service|      Customer Service|                YES|
|          Brush|Solid Waste Manag...|           Solid Waste|                YES|
+---------------+--------------------+----------------------+-------------------+
only showing top 2 rows

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
+---------+----------------+
only showing top 2 rows



(None, None, None)

2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`
    - Inspect your folder structure. What do you notice?

In [5]:
# write data to csv locally
#src.write.csv('sources-csv')

In [6]:
# write data to json locally
#src.write.json('sources-json')

In [7]:
# there is a _SUCCESS folder and another folder with the unique content for each file type

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [8]:
src.printSchema(), dep.printSchema(), case.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



(None, None, None)

##### Case data:
- `case_id` --> string, since it's indentifier
- `case_opened_date`, `case_closed_data`, `SLA_due_date` --> date types
- `case_late` and `case_closed` --> boolean

In [9]:
case = case.withColumn('case_id', col('case_id').cast('string'))

In [10]:
# rename SLA_due_date to case_due_date
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

In [11]:
case = (case
    .withColumn('case_late', col('case_late').cast('boolean'))
    .withColumn('case_closed', col('case_closed').cast('boolean'))
)

In [12]:
case.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



---

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [42]:
(
    case
    .select(
        datediff(current_timestamp(),'case_opened_date').alias('case_age'))
    .filter(col('case_closed') == 'false')
    .sort(desc('case_age'))
    .show(5)
)

+--------+
|case_age|
+--------+
|    1429|
|    1429|
|    1428|
|    1427|
|    1426|
+--------+
only showing top 5 rows



In [13]:
case.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 case_due_date        | 2018-01-05 08:30:00  
 case_late            | false                
 num_days_late        | -2.0126041

2. How many Stray Animal cases are there?

In [14]:
case.filter(col('service_request_type') == 'Stray Animal').count()

26760

3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [15]:
(
    case
    .filter(col('dept_division') == 'Field Operations')
    .filter(col('service_request_type') != 'Officer Standby')
    .count()
)

113902

4. Convert the `council_district` column to a string column.

In [17]:
case = case.withColumn("council_district", col("council_district").cast("string"))

5. Extract the year from the case_closed_date column.

In [50]:
(
    case
    .withColumn('year', year('case_closed_date'))
    .select('year', 'case_closed_date')
    .show(5)
)

+----+-------------------+
|year|   case_closed_date|
+----+-------------------+
|2018|2018-01-01 12:29:00|
|2018|2018-01-03 08:11:00|
|2018|2018-01-02 07:57:00|
|2018|2018-01-02 08:13:00|
|2018|2018-01-01 13:29:00|
+----+-------------------+
only showing top 5 rows



6. Convert num_days_late from days to hours in new columns num_hours_late.

In [52]:
(
    case
    .withColumn('num_hours_late', expr('num_days_late * 24'))
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?

**Bonus**: You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.